In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [2]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape:", x_train.shape)
print("y_train.shape:", y_train.shape)

x_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000,)


In [3]:
# number of classes
K = len(set(y_train))
print("number of classes:", K)

number of classes: 10


In [4]:
print(x_train[0].shape)

(32, 32, 3)


In [5]:
# Build the model using the functional API

# input 
i = Input(shape=x_train[0].shape)

x = Conv2D(32, (3, 3), strides=2, activation='relu')(i)
x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
x = Conv2D(128, (3, 3), strides=2, activation='relu')(x)

# Flatten
x = Flatten()(x)

# Dropout
x = Dropout(0.5)(x)

# Dense(relu)
x = Dense(1024, activation='relu')(x) #dense 많아졌다 -> 가중치도 많아진다 -> 열심히 학습한다. 

# Dropout
x = Dropout(0.2)(x) #드롭아웃은 옵션사항. 

# Dense(softmax)
x = Dense(K, activation='softmax')(x)


model = Model(i,x)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 15, 15, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 1152)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1152)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              118067

In [6]:
# Compile and fit
# Note: make sure you are using hte GPU for this!

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
             metrics=['accuracy']) #sparse : categorical_crossentropy 를 거쳐서 나오는 값을 정수로 바꿔서 출력해라. 

result = model.fit(x_train, y_train,
                  validation_data=(x_test,y_test),
                   epochs=10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 52s 1ms/sample - loss: 1.5656 - accuracy: 0.4288 - val_loss: 1.2681 - val_accuracy: 0.5390
Epoch 2/10
50000/50000 [==============================] - 51s 1ms/sample - loss: 1.2704 - accuracy: 0.5425 - val_loss: 1.1431 - val_accuracy: 0.5886
Epoch 3/10
50000/50000 [==============================] - 52s 1ms/sample - loss: 1.1429 - accuracy: 0.5891 - val_loss: 1.0335 - val_accuracy: 0.6364
Epoch 4/10
50000/50000 [==============================] - 53s 1ms/sample - loss: 1.0519 - accuracy: 0.6238 - val_loss: 0.9696 - val_accuracy: 0.6575
Epoch 5/10
50000/50000 [==============================] - 53s 1ms/sample - loss: 0.9729 - accuracy: 0.6530 - val_loss: 0.9316 - val_accuracy: 0.6684
Epoch 6/10
50000/50000 [==============================] - 51s 1ms/sample - loss: 0.9158 - accuracy: 0.6745 - va

In [12]:
# Fit with data augmentation
# Node: if you run this AFTER calling the previous model.fit(), it will CONTINUE training where it left off
# data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 32
data_generator = ImageDataGenerator(
width_shift_range=0.1,
height_shift_range=0.1,
horizontal_flip=True)

print(x_train.shape[0])
print(x_train.shape[0] // batch_size)


50000
1562


In [15]:
train_generator = data_generator.flow(x_train, y_train, batch_size)
step_per_epochs = x_train.shape[0] // batch_size
result = model.fit_generator(train_generator,
                            validation_data=(x_test, y_test),
                            steps_per_epoch=step_per_epochs,
                            epochs=10)

Epoch 1/10
1562/1562 [==============================] - 80s 51ms/step - loss: 1.0612 - accuracy: 0.6244 - val_loss: 0.8874 - val_accuracy: 0.6956
Epoch 2/10
1562/1562 [==============================] - 79s 51ms/step - loss: 1.0282 - accuracy: 0.6376 - val_loss: 0.9153 - val_accuracy: 0.6795
Epoch 3/10
1562/1562 [==============================] - 80s 51ms/step - loss: 0.9990 - accuracy: 0.6461 - val_loss: 0.8233 - val_accuracy: 0.7199
Epoch 4/10
1562/1562 [==============================] - 78s 50ms/step - loss: 0.9889 - accuracy: 0.6504 - val_loss: 0.8805 - val_accuracy: 0.6931
Epoch 5/10
1562/1562 [==============================] - 80s 51ms/step - loss: 0.9680 - accuracy: 0.6589 - val_loss: 0.8612 - val_accuracy: 0.7075
Epoch 6/10
1562/1562 [==============================] - 93s 59ms/step - loss: 0.9638 - accuracy: 0.6604 - val_loss: 0.8697 - val_accuracy: 0.7025
Epoch 7/10
1562/1562 [==============================] - 91s 58ms/step - loss: 0.9519 - accuracy: 0.6630 - val_loss: 0.8359 -

In [ ]:
# Plot loss per iteration


In [ ]:
# Plot accuracy per iteration


In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes, 
                          normalize=False, 
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be appled by setting `normalize=True`.
  """
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("normalized confusion matrix")
  else:
    print('Consusion matrix, without normalization')
  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
  
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

p_test = model.predict(x_test).argmax(axis=1)
cm = confusion_matrix(y_test, p_test)
plot_confusion_matrix(cm, list(range(10)))

In [ ]:
# Label mapping


In [ ]:
# Show some misclassified examples


In [ ]:
# Now that the model is so large, it's useful to summarie it
